<a href="https://colab.research.google.com/github/rahadiyant/dataku/blob/main/TindakanDokter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [27]:
!pip install streamlit==1.27.0

In [28]:
!pip install pandas
!pip install matplotlib
!pip install scikit-learn
!pip install scipy
!pip install plotly
!pip install openpyxl
!pip install pyngrok

In [29]:
%%writefile doctor_analysis.py
import streamlit as st
import pandas as pd
import plotly.express as px
import numpy as np

# Konfigurasi halaman
st.set_page_config(page_title="Dashboard Laporan Tindakan Dokter Mata", layout="wide")

# Fungsi untuk memuat data
def load_data():
    url = "https://github.com/rahadiyant/dataku/raw/main/LapTindakanDokter.csv"
    df = pd.read_csv(url, encoding='utf-8')

    # Tampilkan nama kolom yang tersedia sebelum diubah
    st.sidebar.write("Kolom yang tersedia (sebelum diubah):", df.columns.tolist())

    # Ubah nama kolom jika diperlukan
    df = df.rename(columns={
        'NAMA DOKTER': 'nama_dokter',
        'JENIS TINDAKAN': 'jenis_tindakan',
        'TANGGAL': 'tanggal',
        'HARGA': 'harga'
    })

    # Konversi kolom tanggal ke tipe datetime
    df['tanggal'] = pd.to_datetime(df['tanggal'], errors='coerce')

    # Bersihkan dan konversi kolom harga
    df['harga'] = df['harga'].str.replace('Total: ', '').str.replace(',', '').astype(float)

    return df

# Memuat data
df = load_data()

# Judul
st.title("Dashboard Laporan Tindakan Dokter Mata")

# Sidebar
st.sidebar.header("Filter")

# Tampilkan nama kolom yang tersedia setelah diubah
st.sidebar.write("Kolom yang tersedia (setelah diubah):", df.columns.tolist())

# Filter data
try:
    dokter = st.sidebar.multiselect("Pilih Dokter", options=df['nama_dokter'].unique())
    tindakan = st.sidebar.multiselect("Pilih Jenis Tindakan", options=df['jenis_tindakan'].unique())
except KeyError as e:
    st.error(f"Kolom tidak ditemukan: {e}. Mohon periksa nama kolom yang benar.")
    st.stop()

if dokter:
    df = df[df['nama_dokter'].isin(dokter)]
if tindakan:
    df = df[df['jenis_tindakan'].isin(tindakan)]

# Tampilkan informasi dasar
st.header("Total Data")
col1, col2, col3 = st.columns(3)
col1.metric("Total Catatan", len(df))
col2.metric("Total Dokter", df['nama_dokter'].nunique())
col3.metric("Total Jenis Tindakan", df['jenis_tindakan'].nunique())

# Grafik jumlah tindakan per dokter
st.subheader("Jumlah Tindakan per Dokter")
dokter_count = df['nama_dokter'].value_counts().reset_index()
dokter_count.columns = ['nama_dokter', 'jumlah_tindakan']
fig_dokter = px.bar(dokter_count,
                    x='nama_dokter', y='jumlah_tindakan',
                    labels={'nama_dokter': 'Nama Dokter', 'jumlah_tindakan': 'Jumlah Tindakan'})
st.plotly_chart(fig_dokter, use_container_width=True)

# Grafik distribusi jenis tindakan
st.subheader("Distribusi Jenis Tindakan")
tindakan_count = df['jenis_tindakan'].value_counts().reset_index()
tindakan_count.columns = ['jenis_tindakan', 'jumlah']
fig_tindakan = px.pie(tindakan_count,
                      values='jumlah', names='jenis_tindakan')
st.plotly_chart(fig_tindakan, use_container_width=True)

# Grafik tren harga tindakan
st.subheader("Tren Harga Tindakan")
harga_trend = df.groupby('tanggal')['harga'].mean().reset_index()
fig_harga = px.line(harga_trend,
                    x='tanggal', y='harga',
                    labels={'tanggal': 'Tanggal', 'harga': 'Rata-rata Harga'})
st.plotly_chart(fig_harga, use_container_width=True)

# Tampilkan data mentah
if st.checkbox("Tampilkan Data Mentah"):
    st.subheader("Data Mentah")
    st.write(df)

# Tampilkan sampel data dan informasi kolom
st.subheader("Sampel Data")
st.write(df.head())
st.subheader("Informasi Kolom")
st.write(df.info())

Overwriting doctor_analysis.py


In [31]:
from pyngrok import ngrok
ngrok.set_auth_token("cr_2iJyVmdEVq1QrJmRETFGsDMgL9W")

In [ ]:
!streamlit run doctor_analysis.py & npx localtunnel --port 8501

[..................] \ fetchMetadata: sill resolveWithNewModule localtunnel@2.0


  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://34.86.156.170:8501

npx: installed 22 in 2.502s
your url is: https://khaki-beds-behave.loca.lt
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15107 entries, 0 to 15106
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype         
---  ------          --------------  -----         
 0   nomor_kwitansi  15106 non-null  object        
 1   tanggal         15106 non-null  datetime64[ns]
 2   pasien          15106 non-null  object        
 3   nama_dokter     15106 non-null  object        
 4   poliklinik      15106 non-null  object        
 5   tipe            15106 non-null  object        
 6   jenis_tindakan  15106 non-null  object        
 7   harga           15107 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(6)
memory usage: 944.3+ KB
